# 🔬 Multi-Magnification Breast Classification using VGG16 + Inception Block by performing Undersampling

### 40X Magnification

In this experiment, histopathology images at 40X magnification were used for binary classification (benign vs. malignant).
Undersampling was applied to balance the classes and avoid model bias toward the malignant class.
Despite the lower resolution, the model showed decent performance in distinguishing patterns.


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

# Load image paths
benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Original Benign Images: {len(benign_images)}")
print(f"Original Malignant Images: {len(malignant_images)}")

# Undersample malignant images to match benign count
malignant_images = resample(malignant_images,
                            replace=False,
                            n_samples=len(benign_images),
                            random_state=42)

print(f"Undersampled Malignant Images: {len(malignant_images)}")

# Assign labels
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine and convert to arrays
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# Create tf.data.Dataset objects
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Define custom Inception block
def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Build model
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Original Benign Images: 626
Original Malignant Images: 1370
Undersampled Malignant Images: 626
Training samples: 750
Validation samples: 126
Testing samples: 376
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 355s 23s/step - accuracy: 0.5027 - loss: 1.0843 - val_accuracy: 0.5000 - val_loss: 0.6969
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.5226 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6787
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.6069 - loss: 0.6621 - val_accuracy: 0.7063 - val_loss: 0.6418
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.6920 - loss: 0.6206 - val_accuracy: 0.6984 - val_loss: 0.6043
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 896ms/step - accuracy: 0.7189 - loss: 0.5830 - val_accuracy: 0.6667 - val_loss: 0.6102
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 943ms/step - accuracy: 0.6834 - loss: 0.5771 - val_accuracy: 0.6984 - val_loss: 0.5924
Epoch 7/100
6/6 ━━━━━━━━━━━━━

### 100X Magnification

At 100X magnification, the dataset provided slightly better structural clarity than 40X.
Balanced class distribution was achieved via undersampling of the malignant images.
Improved texture and cell boundary visibility contributed to better feature extraction.
The model showed increased accuracy and F1 score compared to the 40X experiment.


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

# Load image paths
benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Original Benign Images: {len(benign_images)}")
print(f"Original Malignant Images: {len(malignant_images)}")

# Undersample malignant to match benign count
malignant_images = resample(malignant_images,
                            replace=False,
                            n_samples=len(benign_images),
                            random_state=42)

print(f"After Undersampling - Malignant: {len(malignant_images)}")

# Labels
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine and convert to arrays
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# Prepare datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Inception block
def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    return concatenate([branch1, branch2, branch3, branch4], axis=-1)

# Add Inception to VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Final model
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluation
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)
loss, accuracy = model.evaluate(test_dataset)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Original Benign Images: 649
Original Malignant Images: 1437
After Undersampling - Malignant: 649
Training samples: 778
Validation samples: 130
Testing samples: 390
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 147s 4s/step - accuracy: 0.5003 - loss: 0.9872 - val_accuracy: 0.5000 - val_loss: 0.7139
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 71s 863ms/step - accuracy: 0.5367 - loss: 0.6902 - val_accuracy: 0.5154 - val_loss: 0.6908
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 712ms/step - accuracy: 0.5647 - loss: 0.6763 - val_accuracy: 0.5000 - val_loss: 0.6706
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 867ms/step - accuracy: 0.6135 - loss: 0.6405 - val_accuracy: 0.5846 - val_loss: 0.6449
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 712ms/step - accuracy: 0.6693 - loss: 0.5932 - val_accuracy: 0.6692 - val_loss: 0.5665
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 775ms/step - accuracy: 0.7335 - loss: 0.5225 - val_accuracy: 0.7308 - val_loss: 0.5229
Epoch 7/100
7/7 ━━━

### 200X Magnification

The 200X magnification images offered finer cellular details, aiding deeper feature learning.
Both F1 score and geometric mean significantly improved at this magnification.
It emerged as one of the most effective magnifications for classification tasks.


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Original Benign Images: {len(benign_images)}")
print(f"Original Malignant Images: {len(malignant_images)}")

# Undersample malignant to match benign count
malignant_images = resample(malignant_images,
                            replace=False,
                            n_samples=len(benign_images),
                            random_state=42)

print(f"After Undersampling - Malignant: {len(malignant_images)}")

# Labels
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Add Inception block after VGG16
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Predict
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Metrics
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)
loss, accuracy = model.evaluate(test_dataset)

# Print results
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Original Benign Images: 623
Original Malignant Images: 1390
After Undersampling - Malignant: 623
Training samples: 747
Validation samples: 125
Testing samples: 374
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 157s 14s/step - accuracy: 0.4770 - loss: 1.2523 - val_accuracy: 0.4960 - val_loss: 0.7532
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 840ms/step - accuracy: 0.4913 - loss: 0.7243 - val_accuracy: 0.5040 - val_loss: 0.6997
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5278 - loss: 0.6874 - val_accuracy: 0.5120 - val_loss: 0.6803
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5875 - loss: 0.6667 - val_accuracy: 0.6400 - val_loss: 0.6589
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6665 - loss: 0.6395 - val_accuracy: 0.6240 - val_loss: 0.6323
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 842ms/step - accuracy: 0.7142 - loss: 0.5987 - val_accuracy: 0.7120 - val_loss: 0.5927
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.6986 - loss: 0.

### 400X Magnification

At 400X magnification, the images offered high-resolution details of cell morphology.
Despite the richness of features, performance slightly plateaued compared to 200X.
Careful preprocessing was crucial to avoid overfitting on very localized structures.


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, concatenate, AveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample

# Dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

# Load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Original Benign: {len(benign_images)}")
print(f"Original Malignant: {len(malignant_images)}")

# Undersample malignant to match benign
malignant_images = resample(malignant_images,
                            replace=False,
                            n_samples=len(benign_images),
                            random_state=42)

print(f"After Undersampling - Malignant: {len(malignant_images)}")

# Labels
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split dataset (60% train, 10% val, 30% test)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Image preprocessing
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).map(process_path).shuffle(1000).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).map(process_path).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).map(process_path).batch(BATCH_SIZE)

if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Inception block
def inception_block(x):
    branch1 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)
    return output

# Build model
x = inception_block(base_model.output)
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)
loss, accuracy = model.evaluate(test_dataset)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Original Benign: 588
Original Malignant: 1232
After Undersampling - Malignant: 588
Training samples: 705
Validation samples: 118
Testing samples: 353
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 123s 9s/step - accuracy: 0.4892 - loss: 1.3178 - val_accuracy: 0.5000 - val_loss: 0.7236
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 774ms/step - accuracy: 0.4970 - loss: 0.7091 - val_accuracy: 0.5000 - val_loss: 0.7005
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 782ms/step - accuracy: 0.5190 - loss: 0.6915 - val_accuracy: 0.5000 - val_loss: 0.6823
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 790ms/step - accuracy: 0.5479 - loss: 0.6766 - val_accuracy: 0.7119 - val_loss: 0.6684
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 996ms/step - accuracy: 0.6913 - loss: 0.6604 - val_accuracy: 0.7542 - val_loss: 0.6451
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 780ms/step - accuracy: 0.7347 - loss: 0.6235 - val_accuracy: 0.7119 - val_loss: 0.6172
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 941ms/step - accuracy: 0.6980 - loss: 0.596

2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 30s 13s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 979ms/step - accuracy: 0.8518 - loss: 0.7656
Test Accuracy: 0.8442
F1 Score: 0.8433
G-Mean: 0.8442
Informedness (IBA): 0.6884


In [2]:
import pandas as pd

# Create a DataFrame with the updated metrics
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.8590, 0.8282, 0.8449, 0.8449],
    'F1 Score': [0.8602, 0.8232, 0.8449, 0.8449],
    'G-Mean': [0.8590, 0.8277, 0.8449, 0.8449],
    'Informedness (IBA)': [0.7181, 0.6564, 0.6898, 0.6898]
}

df = pd.DataFrame(data)

# Display the table with formatting
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#000000'), ('font-weight', 'bold')]
    }]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.8590,0.8602,0.8590,0.7181
100X,0.8282,0.8232,0.8277,0.6564
200X,0.8449,0.8449,0.8449,0.6898
400X,0.8449,0.8449,0.8449,0.6898
